In [1]:
import csv
import math
from statistics import median
from DBconnect import myDB
#using Binance
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
import numpy as np

f = open('binance.key', 'r')
client = Client(f.readline().strip(), f.readline().strip(), tld='us')

mydb = myDB()
mydb.load_db('db.key')
mydb.dbconnect()

In [2]:
#Getting All symbols from Binance US
#Get 20 symbols per IP to not get blocked
def getBiSymbols():
    raw_symbols = client.get_all_tickers()
    bi_symbols = []

    for s in range(len(raw_symbols)):
        bi_symbols.append(raw_symbols[s]['symbol'])

    for i in range(3):
        for w in bi_symbols:
            if w.endswith('USD'):
                pass
            else:
                bi_symbols.remove(w)
    for i in range(3):
        for w in bi_symbols:
            if w.endswith('BUSD'):
                bi_symbols.remove(w)
            else:
                pass
    return bi_symbols

In [13]:
def getHLTable(rg):
    hltable=pd.DataFrame()

    #Get 20 symbols per IP to not get blocked
    for bs in range(0,len(rg)):
        klines = client.get_historical_klines(bi_symbols[bs], Client.KLINE_INTERVAL_1WEEK, "1 DEC, 2020")
        df = pd.DataFrame(klines,columns = ['OpenTime','Open','High','Low','Close','Volume','CloseTime','QAV','Trades','TBAV','TQAV','ignore'])
        df = df.iloc[::-1]
        if bs == 0:
            hltable['time'] = df['OpenTime']
        hltable[bi_symbols[bs]+"-Close"] = df['Close']
        hltable[bi_symbols[bs]+"-High"] = df['High']
        hltable[bi_symbols[bs]+"-Low"] = df['Low']


    hltable.dropna(axis='columns', inplace=True)
    return hltable

In [4]:
def getHLCompareTable(hltable):
    hlCompareTable=pd.DataFrame()
    hlCompareTable2=pd.DataFrame()

    #hlCompare Table saves Area, hlcompareTable2 saves High Price 
    tlen = hltable.shape[0]
    for col in hltable.columns:
        currency = col.split('-')[0]
        if col == 'time':
            continue
        elif currency in hlCompareTable.columns and currency in hlCompareTable2.columns:
            continue
        else:
            valueList = np.zeros(shape=(tlen-1,1))
            secondValueList = np.zeros(shape=(tlen-1,1))
            for i in range(tlen-1):
                try:
                    c = float(hltable[currency+'-Close'][tlen-2-i])
                except:
                    print(i)
                h = float(hltable[currency+'-High'][tlen-1-i])
                l = float(hltable[currency+'-Low'][tlen-1-i])

                if pd.isna(c) and pd.isna(h) and pd.isna(l):    
                    break
                else:
                    valueList[i][0] =((h-l)/c*100)
                    secondValueList[i][0] =((h-c)/c*100)
            hlCompareTable[currency] = pd.DataFrame(valueList, columns = [currency])[currency]
            hlCompareTable2[currency] = pd.DataFrame(secondValueList, columns = [currency])[currency]
    return hlCompareTable, hlCompareTable2


In [41]:
# Get table of runners: calculated base price and runner price
# CalcTable saves area difference, CalcTable2 saves high price difference.
def getHLCalcTable(hlCompareTable, hlCompareTable2):
    tlen = hlCompareTable.shape[0]
    hlCalcTable=pd.DataFrame()
    hlCalcTable2=pd.DataFrame()

    for col in hlCompareTable.columns:
        if col == 'BTCUSD':
            pass
        elif col in hlCalcTable.columns and col in hlCalcTable2.columns:
            pass
        else:
            calcList = np.zeros(shape=(tlen-1,1))
            secondCalcList = np.zeros(shape=(tlen-1,1))
            for i in range(tlen-1):
                baseArea = float(hlCompareTable['BTCUSD'][i+1])
                runnerArea = float(hlCompareTable[col][i])

                baseHigh = float(hlCompareTable2['BTCUSD'][i+1])
                runnerHigh = float(hlCompareTable2[col][i])

                calcList[i][0] = baseArea - runnerArea
                secondCalcList[i][0] = baseHigh - runnerHigh

            hlCalcTable[col] = pd.DataFrame(calcList, columns = [col])[col]
            hlCalcTable2[col] = pd.DataFrame(secondCalcList, columns = [col])[col]
    return hlCalcTable, hlCalcTable2

In [46]:
# Get a table of similarly moved runners
def getSimilarTable(hlCalcTable, hlCalcTable2, hlCompareTable, hlCompareTable2):
    tlen = hlCalcTable.shape[0]
    similarTable=pd.DataFrame()
    signalCol = []
    signalArea = []
    signalHigh = []
    resultHigh = []
    signalIndex = []
    for col in hlCalcTable.columns:

        for i in range(tlen):
            if abs(hlCalcTable[col][i]) < 1:
                if hlCalcTable2[col][i] < 1:
                        signalArea.append(round(hlCompareTable['BTCUSD'][i+1],2))
                        signalHigh.append(round(hlCompareTable2['BTCUSD'][i+1],2))
                        resultHigh.append(round(hlCompareTable2[col][i],2))
                        signalIndex.append(i)
                        signalCol.append(col)
    try:
        similarTable['Signal Col'] = pd.Series(signalCol)
        similarTable['Signal Area'] = pd.Series(signalArea)
        similarTable['Signal High'] = pd.Series(signalHigh)
        similarTable['Result High'] = pd.Series(resultHigh)
        similarTable['Signal Index'] = pd.Series(signalIndex)
    except:
            print("An exception occurred")
    similarTable = similarTable.loc[similarTable['Result High'] > 1]
    return similarTable

In [25]:
#Getting Max signaled currency
def getPivotTable(similarTable):
    pivotTable = similarTable.pivot_table(columns=['Signal Col'], aggfunc='size')
    pivotTable.sort_values(ascending=False, inplace =True)
    return pivotTable

def getMaxList(pivotTable):
    maxSignal = pivotTable.max()

    maxSeries = pd.DataFrame(pivotTable.where(pivotTable.eq(maxSignal))).stack()
    maxList = []
    for i in range(maxSeries.size):
        maxList.append(maxSeries.index[i][0])
    return maxList

In [64]:
# Get most high currency among max signaled currencies
def getBuySignal(pivotTable, similarTable, hlCompareTable, hlCompareTable2):
    maxUnit = ''
    buySignal = False
    for m in pivotTable.index:
        signalIndexList = []
        if pivotTable[m] < 5:
            break
        indexList = similarTable.where(similarTable['Signal Col'] == m).dropna().index
        signalHighList = []
        signalAreaList = []
        for i in indexList:
            signalHighList.append(similarTable.where(similarTable['Signal Col'] == m).dropna()['Signal High'][i])
            signalAreaList.append(similarTable.where(similarTable['Signal Col'] == m).dropna()['Signal Area'][i])

        signalEvaluateTable = hlCompareTable[['BTCUSD', m]]
        signalEvaluateTable['BTCUSD-High'] = hlCompareTable2['BTCUSD']
        signalEvaluateTable[m+'-High'] = hlCompareTable2[m]

        for i in range(len(signalHighList)):
            sIndexList = signalEvaluateTable.loc[(signalEvaluateTable['BTCUSD'] > signalAreaList[i]-0.5) & (signalEvaluateTable['BTCUSD'] < signalAreaList[i]+0.5) & (signalEvaluateTable['BTCUSD-High'] < signalHighList[i]+0.5) & (signalEvaluateTable['BTCUSD-High'] < signalHighList[i]+0.5)].index
            for s in sIndexList:
                signalIndexList.append(s)

        #if signalIndexList contains 0 -> buy signal
        # print(set(signalIndexList))
        if signalIndexList.count(0) > 0:
            buySignal = True
            maxUnit = m
            break
        else:
            buySignal = False
    return maxUnit, buySignal, signalIndexList

In [9]:
# Result Median High is expected profit %
def getMed(signalIndexList):
        sigCount = 0
        sigResultList = []
        for idx in signalIndexList:
            try:
                sigResultList.append(hlCompareTable2[maxUnit][idx-1])
                sigCount+=1
            except:
                pass

        med = median(sigResultList)
        return med

def updateDB(maxUnit, buySignal, signalIndexList, target):
    if buySignal:
        print('Signal Unit: ', maxUnit)        
        print('Result median High: ', round(target,2))
        print('Signal Count : ',len(sigResultList))

        mydb.runQuery(maxUnit, round(target,2))

    else:
        print('NO SIGNAL')

In [10]:
#need 2 y data for back testing
bi_symbols = getBiSymbols()
hltable = getHLTable(bi_symbols)
hlCompareTable, hlCompareTable2 = getHLCompareTable(hltable)
hlCalcTable, hlCalcTable2 = getHLCalcTable(hlCompareTable, hlCompareTable2)

#pass 1 y data to back test
similarTable = getSimilarTable(hlCalcTable, hlCalcTable2, hlCompareTable, hlCompareTable2)

pivotTable = getPivotTable(similarTable)

maxList = getMaxList(pivotTable)

maxUnit, buySignal, signalIndexList = getBuySignal(pivotTable, similarTable, hlCompareTable, hlCompareTable2)
#If buySignal = Ture then, check if next_series_hlCompareTable2[0] > target
#If true, result *= target
#If false, result *= 
target = getMed(signalIndexList)
updateDB(maxUnit, buySignal, signalIndexList, target)

NameError: name 'getBisymbols' is not defined

In [66]:
hlCompareTable2

,BTCUSD,ETHUSD,BCHUSD,LTCUSD,USDTUSD,ADAUSD,BATUSD,ETCUSD,XLMUSD,ZRXUSD,...,MKRUSD,DAIUSD,ONEUSD,BANDUSD,STORJUSD,UNIUSD,SOLUSD,EGLDUSD,PAXGUSD,OXTUSD
0,1.118261,1.476972,1.678884,2.908117,0.009997,0.979998,2.464664,1.413849,8.524073,0.751003,...,2.664111,0.080016,2.893236,4.956450,0.734728,1.320348,0.608280,0.455095,0.410955,0.601093
1,0.246573,0.303863,0.272956,1.364952,0.009995,0.786876,3.204328,0.573359,0.253531,0.225975,...,1.313920,0.140014,31.900212,0.937343,11.250737,0.599434,9.530004,31.139409,2.256794,1.704413
2,5.109994,6.950097,10.761739,8.920698,0.009987,7.289327,20.860842,7.902576,8.030726,16.323335,...,6.982066,0.059970,7.688467,11.303158,16.427176,13.666951,2.480592,9.307338,2.379551,14.481555
3,3.333973,11.265650,3.266654,10.271056,8.923753,10.302688,3.129890,7.791896,12.596480,4.913295,...,4.921720,0.199820,2.617801,5.704045,11.981733,14.462246,21.065277,3.640146,1.690973,9.379968
4,1.430287,6.914316,10.453204,3.629559,0.559496,0.185185,79.355198,5.633179,3.541364,12.361459,...,16.525581,0.550275,9.328599,24.457299,77.642276,2.975606,0.628743,27.526429,0.540415,11.636928
5,1.285949,3.129733,0.980594,0.712342,0.169966,2.656472,7.442914,0.906916,4.374337,1.762908,...,7.670243,0.310124,16.182874,5.026018,38.984608,1.749469,3.329188,39.078894,1.321779,0.914634
6,9.055253,5.436819,21.717062,46.075862,0.179856,17.987533,40.010025,20.669931,21.263736,19.012872,...,16.414256,1.859256,9.238950,8.766220,14.067981,12.520894,1.435269,7.419831,4.581207,52.229424
7,4.821459,8.850094,5.936203,9.145865,0.039976,9.065948,12.712461,5.831551,6.913183,22.965337,...,49.645365,0.200060,4.666448,7.240486,6.445890,9.774719,28.264264,21.309765,2.229912,75.988701
8,4.669075,9.256025,2.299762,7.642610,0.179964,3.882625,114.600871,7.264341,4.671141,68.768225,...,4.956973,0.580000,30.370625,14.864336,126.934815,9.689461,8.279527,5.266465,0.789855,8.710721
9,8.879609,13.730775,7.175767,16.824464,0.039996,6.554489,5.440602,12.777064,3.808033,10.245529,...,6.139542,0.150030,39.921294,10.479891,3.419003,7.107765,34.261549,24.856476,2.144931,12.608455
